### Based on instructions at https://jira.lsstcorp.org/browse/DM-22824
### copy QE curves for ts8 camera to lsstcam

In [1]:
import os
import shutil
import glob
import pandas as pd
from lsst.obs.lsst import LsstCamMapper

In [2]:
rname = 'R22'

In [3]:
dlist = ['S00', 'S01', 'S02', 'S10', 'S11', 'S12', 'S20', 'S21', 'S22']

#### Make directories for the curves to go into:

In [4]:
for i in range(len(dlist)):
    my_folder = f"{os.environ['OBS_LSST_DATA_DIR']}/lsstcam/qe_curves/{rname.lower()}_{dlist[i].lower()}"
    if not os.path.exists(my_folder):
        print('creating %s'%my_folder)
        os.makedirs(my_folder)
    else:
        print(' %s exists'%my_folder)

 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s00 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s01 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s02 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s10 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s11 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s12 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s20 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s21 exists
 /home/bxin/lsst_stack/obs_lsst_data/lsstcam/qe_curves/r22_s22 exists


#### We need to copy the appropriate files from ts8 to the corresponding spot in the lsstcam area, then modify the metadata.
#### We combine these two steps into one.

In [5]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
rtmname = dd.rtm.loc[rname]
vendor = dd.vendor[rname]
print(rname, rtmname,vendor)

R22 RTM-024 e2v


In [19]:
# we need to know the sensor index from the name of the sensor
mapper = LsstCamMapper()
cam = mapper.camera

In [27]:
for i in range(len(dlist)):
    srcDir = f"{os.environ['OBS_LSST_DATA_DIR']}/ts8/qe_curves/{rtmname.lower()}_{dlist[i].lower()}"
    destDir = f"{os.environ['OBS_LSST_DATA_DIR']}/lsstcam/qe_curves/{rname.lower()}_{dlist[i].lower()}"
    srcList = glob.glob('%s/*'%srcDir)
    assert len(srcList) == 1
    srcName = srcList[0]
    destName = os.path.join(destDir, os.path.split(srcName)[-1])
    rid = open(srcName, 'r')
    wid = open(destName, 'w')
    detectorId = cam[f'{rname}_{dlist[i]}'].getId()
    for line in rid:
        if 'TS8' in line:
            wid.write('# - {INSTRUME: LSSTCAM}\n')
        elif 'DETECTOR' in line:
            wid.write('# - {DETECTOR: %d}\n'%detectorId)
        elif 'CALIB_ID' in line:
            oldRaftName = line.split(' ')[3][1:]
            oldDector = line.split(' ')[5]
            oldCcd = line.split(' ')[7]
            oldCcdnum = line.split(' ')[8]
            line = line.replace(oldRaftName, 'raftName=%s'%rname)
            line = line.replace(oldDector, 'detector=%d'%detectorId)
            line = line.replace(oldCcd, 'ccd=%d'%detectorId)
            line = line.replace(oldCcdnum, 'ccdnum=%d'%detectorId)
            wid.write(line)
        else:
            wid.write(line)
    rid.close()
    wid.close()